# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'autosaurus', 'classDB', 'config', 'epa', 'fruits_db', 'local', 'met', 'petsitly_marketing', 'uk_food']


In [4]:
# assign the uk_food database to a variable name
db = mongo.uk_food

In [5]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [6]:
# assign the collection to a variable
establishments = db.establishments

In [7]:
# review a document in the establishments collection
pprint(establishments.find_one())

{'AddressLine1': '1 The Droveway',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'FirstLight Bar & Café',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.64041272672,
 'FHRSID': 1290215,
 'LocalAuthorityBusinessID': 'PI/000083029',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DH',
 'RatingDate': '2020-11-09T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6421e9e5bdefb14b3b5e940d'),
 'geocode': {'latitude': '51.150625', 'longitude': '1.378231'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/1290215',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName": "Penang Flavours",
    "BusinessType": "Halal",
    "AddressLine1": "Greenwich"
}

In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
new_inserted = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(new_inserted)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Halal',
 '_id': ObjectId('6421e9fd6a04b94bf9aa56ad')}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
fields = {"BusinessTypeID": 1, "BusinessType": 1}
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
results = establishments.find(query, fields)
first_result = results[0]
for result in results:
    pprint(result)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e940d')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e940e')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e940f')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e9411')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e9412')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e9413')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e9417')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e9419')}
{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9e5bdefb14b3b5e

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one({"_id": new_inserted["_id"]}, {
    "$set": {
        "BusinessTypeID": first_result["BusinessTypeID"],
        "BusinessType": first_result["BusinessType"]
    }
})

In [13]:
# Confirm that the new restaurant was updated
new_inserted = establishments.find_one({"BusinessName": "Penang Flavours"})
pprint(new_inserted)

{'AddressLine1': 'Greenwich',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6421e9fd6a04b94bf9aa56ad')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({"LocalAuthorityName": {"$eq": "Dover"}})

994

In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({"LocalAuthorityName": {"$eq": "Dover"}})

In [16]:
# Check if any remaining documents include Dover
establishments.count_documents({"LocalAuthorityName": {"$regex": "Dover"}})

0

In [17]:
# Check that other documents remain with 'find_one'
pprint(establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6421e9e5bdefb14b3b5e96f3'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude
establishments.update_many({}, [ {"$set": {"geocode.longitude": {"$toDecimal": "$geocode.longitude"}}} ])

In [19]:
# Change the data type from String to Decimal for latitude
establishments.update_many({}, [ {"$set": {"geocode.latitude": {"$toDecimal": "$geocode.latitude"}}} ])

In [20]:
# Check that the coordinates are now numbers
pprint(establishments.find_one({}, {"geocode.latitude": 1, "geocode.longitude": 1}))

{'_id': ObjectId('6421e9e5bdefb14b3b5e96f3'),
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')}}
